In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_bootstrap_components as dbc
from dash import html
from dash import dcc
from dash import Dash, dash_table
from dash.dependencies import Input, Output, State


app = dash.Dash(__name__)
app.config.suppress_callback_exceptions = True


class stock_comp:
    def __init__(self,icon,past):
        
        self.icon = icon

        if past:
            self.past = int(past)
        else:
            self.past = 0

        #self.past = past
        
        company = yf.Ticker(icon)
        company_data = company.history(period="max")

        company_data.reset_index(inplace = True)

        df_company = company_data.tail(self.past)

        df_company.drop(['Open','High','Low','Volume','Stock Splits','Dividends'], axis = 1, inplace = True)

        df_company.rename(columns = {'Close':icon}, inplace = True)

        self.df_company = df_company

    



app = Dash(external_stylesheets=[dbc.themes.SLATE])


app.layout = html.Div(
    children=[
    html.H1('Stocks comparison(Closing price)', style={'textAlign': 'center', 'color': 'rgb(80, 200, 150)', 'font-size': 30,'margin-top':10}),

    html.Br(),

    

    html.Div([
        #html.P('Stock 1:', style = {'font-size':20,'margin-left': 60, 'margin-right': 5,'margin-top':9, 'color':'blue'}),
        html.Div(dcc.Input(id = 'input-s1', placeholder='Enter a stock ticker.',type='text',value='AAPL',style = {'width':700, 'height':35,'margin-left':30}))
        ],style={'display':'flex', 'flex-direction':'row' ,'align-items':'center'}),

    html.Div([
        #html.P('Stock 2:', style = {'font-size':20, 'margin-left': 60 ,'margin-right': 5, 'margin-top':9, 'color':'red'}),
        html.Div(dcc.Input(id = 'input-s2', placeholder='Enter a stock ticker.',type='text',value='GOOGL',style = {'width':700, 'height': 35, 'margin-right': 28,'margin-left':30,})),
        html.P('History:', style = {'font-size':16, 'margin-right': 5, 'margin-top': 9}),
        html.Div(dcc.Input(id = 'input-past',placeholder='Enter history (days ago)',type='text',value='90',style = {'width':175, 'height': 35}))
        ],style={'display':'flex', 'flex-direction':'row' ,'align-items':'center'}),



    html.Div([
    
    html.Div([
    
    html.Div( dbc.Card( dbc.CardBody(html.Div([], id='plot1') )),style = {'margin-bottom':10, 'margin-right':18, 'margin-left':20} ),

    html.Div([
    
    html.Div([
    
    html.H5('Correlation Matrix', style = {'margin-left':20, 'color':'white'}),
    html.Div( dbc.Card([ dbc.CardBody(dash_table.DataTable(id = 'corr', style_table={'color':'black'}, style_header={'backgroundColor': 'rgb(30, 30, 30)', 'color': 'white'}, style_data={'backgroundColor': 'rgb(50, 50, 50)', 'color': 'rgb(80, 200, 150)'} ))] ),
              style = {'margin-right':18, 'margin-left':20}),

    ]),

    html.Div([
    
    html.H5('Latest Closing Price(USD)', style = {'color':'white'}),
    html.Div( dbc.Card(  dbc.CardBody([html.H6(id = 's1'), 
                                       html.Div(id = 'price1',style = {'font-size':30,'text-align':'center','vertical-align':'top'}),
                                       html.Div(id = 'ind1', style = {'width':60,'vertical-align':'top'}), 
                                                 
                                      ])
                       ))
    
    ], style = {'margin-right':18,'width':'100%','height':'100%'}),


    html.Div([
    
    
    html.Div([dbc.Card(dbc.CardBody([html.H6(id = 's2'), 
                                     html.Div(id = 'price2',style = {'font-size':30,'text-align':'center'}),
                                     html.Div(id = 'ind2',style = {'width':60,'vertical-align':'top'})
                                     ])

                       )], style = {'margin-top':32})
    ], style = {'margin-right':18,'width':'100%','height':'100%'})
    
    
    
    ],style = {'display':'flex'}),

    
    ]),



    html.Div([
    
    html.Div( dbc.Card( dbc.CardBody(dash_table.DataTable(id = 'df', style_table={'overflowX': 'scroll', 'height': '600px', 'color':'black'},
                                                          style_header={'backgroundColor': 'rgb(30, 30, 30)', 'color': 'white'}, style_data={'backgroundColor': 'rgb(50, 50, 50)', 'color': 'rgb(80, 200, 150)'},  ))), style = {'margin-right':20}  )
    ]),

     

    ], style = { 'display': 'grid', 'grid-template-columns': '2.5fr 1.5fr' } ),

  
    
    

    ]
)






@app.callback(
        [Output(component_id='plot1', component_property='children'),
         Output(component_id='corr', component_property='data'),
         Output(component_id='df', component_property='data'),
         Output(component_id='s1', component_property='children'),
         Output(component_id='s2', component_property='children'),
         Output(component_id='price1', component_property='children'),
         Output(component_id='price2', component_property='children'),
         Output(component_id='ind1', component_property='children'),
         Output(component_id='ind2', component_property='children'),
        ],
           

        [Input(component_id='input-s1', component_property='value'),
         Input(component_id='input-s2', component_property='value'),
         Input(component_id='input-past', component_property='value')],
        
        [State("plot1", 'children'),State("corr", 'data'),State("df", 'data'),State("s1",'children'),State("s2",'children')
        ,State("price1",'children'),State("price2",'children'),State("ind1",'children'),State("ind2",'children')]
              
)

def stock_results(ticker1,ticker2,days,c1,c2,c3,c4,c5,c6,c7,c8,c9):

    stock1 = stock_comp(ticker1,days)
    stock2 = stock_comp(ticker2,days)

    dataframe = pd.merge(stock1.df_company, stock2.df_company)

    

    # dataframe.set_index('Date',inplace = True)

    correlation = dataframe.corr()

    price1 = '{:.2f}'.format(dataframe.loc[dataframe.index[-1],ticker1])
    price2 = '{:.2f}'.format(dataframe.loc[dataframe.index[-1],ticker2])
    
    graph = px.line(dataframe, x = 'Date' , y = [ticker1,ticker2], title='Value of stocks over time').update_layout(
        template='plotly_dark',
        plot_bgcolor= 'rgba(0, 0, 0, 0)',
        paper_bgcolor= 'rgba(0, 0, 0, 0)',)
    
    ind1 = go.Figure(
        go.Indicator(mode = 'delta', value = float(price1), delta = {'reference':dataframe.loc[dataframe.index[-2],ticker1],'relative':False,'valueformat':'.2f'})
    )
    ind1.update_layout(margin = dict(l=0, r=0, t=0,b=0),
                       template='plotly_dark',
                       plot_bgcolor= 'rgba(0, 0, 0, 0)',
                       paper_bgcolor= 'rgba(0, 0, 0, 0)',
                       height = 30)
    

    ind2 = go.Figure(
        go.Indicator(mode = 'delta', value = float(price2), delta = {'reference':dataframe.loc[dataframe.index[-2],ticker2],'relative':False,'valueformat':'.2f'})
    )
    
    ind2.update_layout(margin = dict(l=0, r=0, t=0,b=0,),
                       template='plotly_dark',
                       plot_bgcolor= 'rgba(0, 0, 0, 0)',
                       paper_bgcolor= 'rgba(0, 0, 0, 0)',
                       height = 30)


    return [dcc.Graph(figure = graph), correlation.to_dict(orient='records'), dataframe.to_dict(orient='records'),
            ticker1,ticker2,price1,price2,dcc.Graph(figure = ind1),dcc.Graph(figure = ind2)]



if __name__ == '__main__':
    app.run_server( host="localhost", debug=False, dev_tools_ui=False, dev_tools_props_check=False) 

Dash is running on http://localhost:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8050/ (Press CTRL+C to quit)
